In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, connections, Index, Document, Date, Nested, Boolean, \
    analyzer, InnerDoc, Completion, Keyword, Text

# Configure connection

In [2]:
connections.create_connection(alias='myconn', hosts=['localhost'], timeout=60)

<Elasticsearch([{'host': 'localhost'}])>

In [3]:
s = Search(using='myconn')

# Drop, create index

In [4]:
blogs = Index('blogs')

# define custom settings
blogs.settings(
    number_of_shards=1,
    number_of_replicas=0
)
# define aliases
blogs.aliases(
    old_blogs={}
)

# You can attach custom analyzers to the index
html_strip = analyzer('html_strip',
    tokenizer="standard",
    filter=["lowercase", "stop", "snowball"],
    char_filter=["html_strip"]
)
blogs.analyzer(html_strip)

# register a document with the index
#blogs.document(Post)

# can also be used as class decorator when defining the Document
@blogs.document
class Post(Document):
    title = Text()
    title_suggest = Completion()
    created_at = Date(default_timezone='UTC')
    published = Boolean()
    category = Text(
        analyzer=html_strip,
        fields={'raw': Keyword()}
    )
    
    def save(self, **kwargs):
        self.created_at = datetime.now()
        return super().save(**kwargs)


# delete the index, ignore if it doesn't exist
blogs.delete(ignore=404, using="myconn")

# create the index in elasticsearch
blogs.create(using="myconn")

/Users/ruhong/work/data/query_class/venv/lib/python3.7/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'blogs'}

In [5]:
# instantiate the document
first = Post(title='My First Blog Post, yay!', published=True)
# assign some field values, can be values or lists of values
first.category = ['everything', 'nothing']
# every document has an id in meta
first.meta.id = first.title
print(f"id={first.meta.id}")

# save the document into the cluster
first.save(using="myconn")
# retrieve the document
p = Post.get(id=first.title, using="myconn")
print(repr(p))

id=My First Blog Post, yay!
Post(index='blogs', id='My First Blog Post, yay!')


In [6]:
# by calling .search we get back a standard Search object
s = Post.search(using="myconn")
# the search is already limited to the index and doc_type of our document
s = s.filter('term', published=True).query('match', title='first')
results = s.execute()
# when you execute the search the results are wrapped in your document class (Post)
for post in results:
    print(f"score={post.meta.score}, title={post.title}")

In [7]:
p.delete(using="myconn")
Post.get(id=first.title, using="myconn", ignore=404)  # suppress NotFound exception 

# References

- https://elasticsearch-dsl.readthedocs.io/en/latest/configuration.html
- https://elasticsearch-dsl.readthedocs.io/en/latest/persistence.html
- https://elasticsearch-dsl.readthedocs.io/en/latest/api.html